In [1]:
# Install required libraries
!pip install requests pandas openpyxl tqdm

In [4]:
import os
import requests
import pandas as pd
import re
import time
from tkinter import Tk, filedialog
from openpyxl import load_workbook
from openpyxl.styles import Font
from tqdm import tqdm

def get_access_token():
    url = "https://login.microsoftonline.com/5b973f99-77df-4beb-b27d-aa0c70b8482c/oauth2/token"
    payload = {
        'grant_type': 'client_credentials',
        'client_secret': 'ZWFmYWY2ZjUtNTFlMy00N2ZlLWE2OWItNGRlNmNkOTlkZjEy',
        'client_id': 'a82f3991-32cc-4a83-adec-2fd572eb5d9a'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, headers=headers, data=payload)
    return response.json().get('access_token')

def extract_with_layout(pdf_path, access_token):
    url = "https://we-api.di.ey.com/v2/di-prod/process/extraction/sync"
    payload = {
        'configuration': '{"process": {"per_document": { "provider_model_id": "prebuilt-layout", "provider": "Microsoft Document Intelligence", "provider_version": "2023-07-31"}}}'
    }
    files = [('file', ('placeholder.pdf', open(pdf_path, 'rb'), 'application/pdf'))]
    headers = {
        'Authorization': f'Bearer {access_token}',
        'x-id-tenant': 'documentsrvcsg',
        'x-name-tenant': 'documentsrvcsg'
    }

    # Polling mechanism
    max_attempts = 10
    for attempt in range(max_attempts):
        response = requests.post(url, headers=headers, data=payload, files=files)
        if response.status_code == 200:
            return response.json()
        print(f"Attempt {attempt + 1} failed. Retrying in 5 seconds...")
        time.sleep(5)  # Wait for 5 seconds before retrying

    print(f"Failed to process {pdf_path}. Status code: {response.status_code}, Response: {response.text}")
    return None

def identify_bank(content):
    if "DBS" in content:
        return "DBS"
    elif "UOB" in content:
        return "UOB"
    elif "HSBC" in content:
        return "HSBC"
    else:
        return "Citi"

def parse_document_citi(result):
    account_data = {
        "account_numbers": [],
        "balances": [],
        "currency": []
    }
    content = result["response"]["result"]["data"]["content"]
    print("Raw Content: ", content)

    account_numbers_found = re.findall(r'\b(\d{5,})\b\s[A-Z]{3}\s\b', content)
    account_data["account_numbers"].extend(account_numbers_found)

    balances_found = re.findall(r'\b[\d,]+\.\d{2}\b', content)
    account_data["balances"].extend(balances_found)

    currency_found = re.findall(r'\b\d{5,}\b\s([A-Z]{3})\s\b', content)
    account_data["currency"].extend(currency_found)
    while len(account_data["account_numbers"]) > len(account_data["currency"]):
        account_data["currency"].append("Not Found")
        
    print("Extracted Account Numbers: ", account_data["account_numbers"])
    print("Extracted Balances: ", account_data["balances"])
    print("Extracted Currencies: ", account_data["currency"])

    return account_data, content

def match_accounts_citi(ocr_data, content, results, pdf_file_path):
    file_name = os.path.basename(pdf_file_path)
    if not ocr_data["account_numbers"] or not ocr_data["balances"]:
        print(f"Manual check required for {pdf_file_path}: No account number or balance noted.")
        results.append({
            "Account_Number": "Missing",
            "Currency": "Missing",
            "OCR_Balance": "Missing",
            "Associated_File": f'=HYPERLINK("{pdf_file_path}", "{file_name}")',
        })
        return
        
    # Ensure the matching is done based on the shortest list
    max_length = min(len(ocr_data["account_numbers"]), len(ocr_data["balances"]), len(ocr_data["currency"]))
        
    for idx in range(max_length):
        account_number = ocr_data["account_numbers"][idx]
        ocr_balance = ocr_data["balances"][idx].replace(',', '') if idx < len(ocr_data["balances"]) else "Not Found"
        currency = ocr_data["currency"][idx] if idx < len(ocr_data["currency"]) else "Not Found"
        hyperlink = f'=HYPERLINK("{pdf_file_path}", "{file_name}")'
        print(f"Matched Account: {account_number}, Currency: {currency}, OCR Balance: {ocr_balance}, Confidence: 100%")
        results.append({
            "Account_Number": account_number,
            "Currency": currency,
            "OCR_Balance": ocr_balance,
            "Associated_File": hyperlink
        })

def parse_document_uob(result):
    account_data = {
        "account_numbers": [],
        "balances": [],
        "currency": []
    }
    content = result["response"]["result"]["data"]["content"]
    print("Raw Content: ", content)

    account_numbers_found = re.findall(r'\b(\d{1,}-\d{3,}-\d{1,}-?\d{0,})\b', content)
    account_data["account_numbers"].extend(account_numbers_found)

    balances_found = re.findall(r'\b[\d,]+\.\d{2}\b|NIL', content)
    account_data["balances"].extend(balances_found)

    currency_found = re.findall(r'\b\d{1,}-\d{3,}-\d{1,}-?\d{0,}\s([A-Z]{3})\b', content)
    account_data["currency"].extend(currency_found)
    while len(account_data["account_numbers"]) > len(account_data["currency"]):
        account_data["currency"].append("Not Found")

    print("Extracted Account Numbers: ", account_data["account_numbers"])
    print("Extracted Balances: ", account_data["balances"])
    print("Extracted Currencies: ", account_data["currency"])  

    return account_data, content

def match_accounts_uob(ocr_data, content, results, pdf_file_path):
    file_name = os.path.basename(pdf_file_path)
    if not ocr_data["account_numbers"] or not ocr_data["balances"]:
        print(f"Manual check required for {pdf_file_path}: No account number or balance noted.")
        results.append({
            "Account_Number": "Missing",
            "Currency": "Missing",
            "OCR_Balance": "Missing",
            "Associated_File": f'=HYPERLINK("{pdf_file_path}", "{file_name}")',
        })
        return
        
    # Ensure the matching is done based on the shortest list
    max_length = min(len(ocr_data["account_numbers"]), len(ocr_data["balances"]), len(ocr_data["currency"]))
        
    for idx in range(max_length):
        account_number = ocr_data["account_numbers"][idx]
        ocr_balance = ocr_data["balances"][idx].replace(',', '') if idx < len(ocr_data["balances"]) else "Not Found"
        currency = ocr_data["currency"][idx] if idx < len(ocr_data["currency"]) else "Not Found"
        hyperlink = f'=HYPERLINK("{pdf_file_path}", "{file_name}")'
        print(f"Matched Account: {account_number}, Currency: {currency}, OCR Balance: {ocr_balance}, Confidence: 100%")
        results.append({
            "Account_Number": account_number,
            "Currency": currency,
            "OCR_Balance": ocr_balance,
            "Associated_File": hyperlink
        })

def parse_document_dbs(result):
    account_data = {
        "account_numbers": [],
        "balances": [],
        "currencies": []
    }
    content = result["response"]["result"]["data"]["content"]
    print("Raw Content: ", content)

    account_numbers_found = re.findall(r'\(No. (\d{1,}-\d{3,}-\d{1,}-?\d{0,})\)', content)
    account_data["account_numbers"].extend(account_numbers_found)

    balances_found = re.findall(r'\b[\d,]+\.\d{2}\b|\bNIL\b', content)
    account_data["balances"].extend(balances_found)

    currencies_found = re.findall(r'([A-Z]{3})\s\b[\d,]+\.\d{2}\b|\bNIL\b', content)
    account_data["currencies"].extend(currencies_found)
    while len(account_data["account_numbers"]) > len(account_data["currencies"]):
        account_data["currencies"].append("Not Found")

    print("Extracted Account Numbers: ", account_data["account_numbers"])
    print("Extracted Balances: ", account_data["balances"])
    print("Extracted Currencies: ", account_data["currencies"])

    return account_data, content

def match_accounts_dbs(ocr_data, content, results, pdf_file_path):
    file_name = os.path.basename(pdf_file_path)
    if not ocr_data["account_numbers"] or not ocr_data["balances"]:
        print(f"Manual check required for {pdf_file_path}: No account number or balance noted.")
        results.append({
            "Account_Number": "Missing",
            "Currency": "Missing",
            "OCR_Balance": "Missing",
            "Associated_File": f'=HYPERLINK("{pdf_file_path}", "{file_name}")',
        })
        return

    account_numbers = ocr_data["account_numbers"]
    balances = ocr_data["balances"]
    currencies = ocr_data["currencies"]

    new_rows = []
    found_accounts = re.findall(r'\(No. (\d{1,}-\d{3,}-\d{1,}-?\d{0,})\)', content)
    lines = content.splitlines()
    last_account_number = None

    if len(balances) > len(account_numbers):
        for line in lines:
            balance_match = re.search(r'\b[\d,]+\.\d{2}\b|\bNIL\b', line)
            if balance_match:
                balance = balance_match.group(0).replace(',', '')
                matched_account = last_account_number if last_account_number else "N/A"
                hyperlink = f'=HYPERLINK("{pdf_file_path}", "{file_name}")'

                # Determine the currency
                currency_index = len(new_rows)  # Current index for new rows
                currency = currencies[currency_index] if currency_index < len(currencies) else "Not Found"

                new_row = {
                    "Account_Number": matched_account,
                    "OCR_Balance": balance,
                    "Currency": currency,
                    "Associated_File": hyperlink,
                }
                new_rows.append(new_row)
                print(f"Matched Account: {matched_account}, Currency: {currency}, OCR Balance: {balance}, Confidence: 100%")

            account_match = re.search(r'\(No. (\d{1,}-\d{3,}-\d{1,}-?\d{0,})\)', line)
            if account_match:
                last_account_number = account_match.group(1)

        for balance in balances[len(new_rows):]:
            new_row = {
                "Account_Number": "N/A",
                "OCR_Balance": balance,
                "Currency": "Not Found",
                "Associated_File": hyperlink,
            }
            new_rows.append(new_row)

    else:
        for i in range(min(len(account_numbers), len(balances))):
            hyperlink = f'=HYPERLINK("{pdf_file_path}", "{file_name}")'
            currency = currencies[i] if i < len(currencies) else "Not Found"
            new_row = {
                "Account_Number": account_numbers[i],
                "OCR_Balance": balances[i],
                "Currency": currency,
                "Associated_File": hyperlink,
            }
            new_rows.append(new_row)
            print(f"Matched Account: {account_numbers[i]}, OCR Balance: {balances[i]}, Currency: {currency}, Confidence: 100%")

        for balance in balances[len(account_numbers):]:
            new_row = {
                "Account_Number": "N/A",
                "OCR_Balance": balance,
                "Currency": "Not Found",
                "Associated_File": hyperlink,
            }
            new_rows.append(new_row)

    for row in new_rows:
        results.append(row)

def parse_document_hsbc(result):
    account_data = {
        "account_numbers": [],
        "balances": [],
        "currency": []
    }
    content = result["response"]["result"]["data"]["content"]
    print("Raw Content: ", content)

    account_numbers_found = re.findall(r'\b(\d{1,}-\d{3,}-\d{1,}-?\d{0,})\b', content)
    account_data["account_numbers"].extend(account_numbers_found)

    balances_found = re.findall(r'^[A-Z]{3}\s(\-{,1}[\d,]+\.{,1}\d{2}|0{1})\b', content, re.MULTILINE)
    account_data["balances"].extend(balances_found)

    currency_found = re.findall(r'^([A-Z]{3})\s(\-{,1}[\d,]+\.{,1}\d{2}|0{1})\b', content, re.MULTILINE)
    account_data["currency"].extend([match[0] for match in currency_found])
    while len(account_data["account_numbers"]) > len(account_data["currency"]):
        account_data["currency"].append("Not Found")

    print("Extracted Account Numbers: ", account_data["account_numbers"])
    print("Extracted Balances: ", account_data["balances"])
    print("Extracted Currencies: ", account_data["currency"])

    return account_data, content

def match_accounts_hsbc(ocr_data, content, results, pdf_file_path):
    file_name = os.path.basename(pdf_file_path)
    if not ocr_data["account_numbers"] or not ocr_data["balances"]:
        print(f"Manual check required for {pdf_file_path}: No account number or balance noted.")
        results.append({
            "Account_Number": "Missing",
            "Currency": "Missing",
            "OCR_Balance": "Missing",
            "Associated_File": f'=HYPERLINK("{pdf_file_path}", "{file_name}")',
        })
        return

    # Ensure the matching is done based on the shortest list
    max_length = min(len(ocr_data["account_numbers"]), len(ocr_data["balances"]), len(ocr_data["currency"]))
        
    for idx in range(max_length):
        account_number = ocr_data["account_numbers"][idx]
        ocr_balance = ocr_data["balances"][idx].replace(',', '') if idx < len(ocr_data["balances"]) else "Not Found"
        currency = ocr_data["currency"][idx] if idx < len(ocr_data["currency"]) else "Not Found"
        hyperlink = f'=HYPERLINK("{pdf_file_path}", "{file_name}")'
        print(f"Matched Account: {account_number}, Currency: {currency}, OCR Balance: {ocr_balance}, Confidence: 100%")
        results.append({
            "Account_Number": account_number,
            "Currency": currency,
            "OCR_Balance": ocr_balance,
            "Associated_File": hyperlink
        })

def auto_fit_columns(output_path):
    workbook = load_workbook(output_path)
    sheet = workbook.active

    for column in sheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)  # Add some padding
        sheet.column_dimensions[column[0].column_letter].width = adjusted_width

    # Apply hyperlink formatting
    for row in sheet.iter_rows(min_row=2, min_col=4, max_col=4):  # Assuming Associated_File is in the 4th column
        for cell in row:
            cell.font = Font(color="0000FF", underline="single")  # Blue color and underline

    # Auto-fit the hyperlink column again
    hyperlink_column = 4  # Assuming the hyperlink is in the 3rd column
    max_length = 0
    column = [cell for cell in sheet.iter_rows(min_row=2, min_col=hyperlink_column, max_col=hyperlink_column)]
    for cell in column:
        try:
            if len(str(cell[0].value)) > max_length:
                max_length = len(str(cell[0].value))
        except:
            pass
    adjusted_width = (max_length + 2)  # Add some padding
    sheet.column_dimensions[sheet.cell(row=1, column=hyperlink_column).column_letter].width = adjusted_width

    workbook.save(output_path)

def main():
    Tk().withdraw()
    folder_path = filedialog.askdirectory(title="Select Folder Containing PDF Files")
    if not folder_path:
        print("Invalid selection")
        return

    access_token = get_access_token()
    if not access_token:
        print("Failed to retrieve access token.")
        return

    results = []  # List to hold all account numbers and balances
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

    # Process each PDF file in the selected folder with progress indication
    for pdf_file in tqdm(pdf_files, desc="Processing PDFs", unit="file"):
        pdf_file_path = os.path.join(folder_path, pdf_file)
        result = extract_with_layout(pdf_file_path, access_token)

        if result is None:
            continue  # Skip to the next file if processing failed

        content = result["response"]["result"]["data"]["content"]
        bank_name = identify_bank(content)

        if bank_name == "Citi":
            parsed_data, content = parse_document_citi(result)
            match_accounts_citi(parsed_data, content, results, pdf_file_path)
        elif bank_name == "UOB":
            parsed_data, content = parse_document_uob(result)
            match_accounts_uob(parsed_data, content, results, pdf_file_path)
        elif bank_name == "DBS":
            parsed_data, content = parse_document_dbs(result)
            match_accounts_dbs(parsed_data, content, results, pdf_file_path)
        elif bank_name == "HSBC":
            parsed_data, content = parse_document_hsbc(result)
            match_accounts_hsbc(parsed_data, content, results, pdf_file_path)

    # Create a DataFrame from the results and save to Excel
    results_df = pd.DataFrame(results)
    output_path = os.path.join(folder_path, "results.xlsx")
    results_df.to_excel(output_path, index=False)
    auto_fit_columns(output_path)

if __name__ == "__main__":
    main()


Processing PDFs:  12%|████████▏                                                        | 1/8 [00:44<05:10, 44.34s/file]

Raw Content:  Our Ref: 202405090108 Your Ref: 09 May 2024
ERNST & YOUNG Singapore
Dear Sir(s)
AUDIT CONFIRMATION FOR TP INNOVATION HOLDINGS PTE LTD
As requested, we are pleased to furnish the following information on the above-captioned as at 31 Mar 2024.
S$ CURRENT ACCOUNT (No. 029-015028-1)
SGD
CC101
XDBS
Page 1 of 1
532,630.31 CC100
For queries, please call your Relationship Manager or our DBS BusinessCare officers. They can be reached at 1800 222 2200 (Local) or +65 6222 2200 (Calls from overseas) between 8:30 am to 8:30 pm on Mondays to Fridays, excluding public holidays.
Yours faithfully,
AUDIT CONFIRMATION TEAM
This is a system-generated letter. No signature is required.
06-06-155 (01/2011) Co. Reg. No. 195800306E GST Reg. No. MR-8500180-3
PDS_ACCPREPLY_SPE_X_0d5801210000003c_00028
Extracted Account Numbers:  ['029-015028-1']
Extracted Balances:  ['532,630.31']
Extracted Currencies:  ['Not Found']
Matched Account: 029-015028-1, OCR Balance: 532,630.31, Currency: Not Found, Confi

Processing PDFs:  25%|████████████████▎                                                | 2/8 [01:24<04:12, 42.10s/file]

Raw Content:  OCBC
I
OCBC Bank 65 Chulia Street OCBC Centre Singapore 049513
Reference: 20250222-OBSAU00136178504 22 February 2025
Private & Confidential ERNST & YOUNG LLP Attention To: CARINE PHONG :selected:
Customer service
+65 6538 1111
8am to 8pm, Monday to Friday
(excluding public holidays)
Audit Confirmation for BORREGAARD S.E.A. PTE LTD
Your Ref
EPE/E-67828187/KY/CP
Our Ref
SG/AC/AUC-203607-01
Dear Sir
We are pleased to append below information in relation to the above customer as at close of business day on 31 December 2024 :-
. Account Balances Balance in Current Account No. 514-047448-001 : SGD58,019.98Cr
This reply is given solely for the purpose of your audit without any responsibility on the part of the Bank or its employees. Please note that we are not in a position to respond to the questions in your letter.
Please feel free to contact our Customer Service Officers at 6538 1111 if you need further clarification.
Yours sincerely
Billy Kueh Head Account Services
Oversea-C

Processing PDFs:  38%|████████████████████████▍                                        | 3/8 [02:06<03:29, 41.98s/file]

...

Raw Content:  HSBC
Customer Request ID : i123bacefg345
Confirmation of Balances as at : 2024-05-31 Customer Number : 012-34567
Account Number
Product Type
Reference
Start Date
Due Date
Currency
Account Balance
012-345678-123
BUSINESSVANTAGE W/O OD
20190101
GBP
9,123,456.78
999-888777-666
TIME DEPOSIT
10M
20220202
20250101
SGD
34,567.78
234-567890-123
CUI - NCI
20240108
EUR
7,654,321.00
Securities Owned
ABC OF SGD999,999,000 A/C: 666-333333-000(OBTAINED)
Extracted Account Numbers:  ['012-345678-123', '999-888777-666', '234-567890-123', '666-333333-000']
Extracted Balances:  ['9,123,456.78', '34,567.78', '7,654,321.00']
Extracted Currencies:  ['GBP', 'SGD', 'EUR', 'Not Found']
Matched Account: 012-345678-123, Currency: GBP, OCR Balance: 9123456.78, Confidence: 100%
Matched Account: 999-888777-666, Currency: SGD, OCR Balance: 34567.78, Confidence: 100%
Matched Account: 234-567890-123, Currency: EUR, OCR Balance: 7654321.00, Confidence: 100%


Processing PDFs:  88%|████████████████████████████████████████████████████████▉        | 7/8 [04:54<00:41, 41.78s/file]

Raw Content:  UOB
EYTEST1234 ERNST & YOUNG LLP ROBINSON ROAD P. O. BOX 384 SINGAPORE 900734
Private And Confidential
United Overseas Bank Limited Cash Management Operations Centre Audit Confirmation Robinson Road PO Box 628 Singapore 901228 www.uobgroup.com
Your Ref.
Our Ref.
Our Reply Date
ASU / ABC / 999999 / EE / ZJZ / 11A
SS 12345678_eeeeebbbbb_A
12-02-2025
Dear Sir,
Confirmation of Balances TEST COMPANY 123
We are pleased to provide the following information as at 31-11-2022
Current Account
222-222-333-1 SGD : 17.00 C
450-322-222-0 SGD
: 3.51C
555-111-77 7-9 GBP : 2,2222.22 C
3-444-9 SGD
: 2,222.77 C
777-321-111-0 USD : 1,111.22 C
222-040-567-9 MYR
: 7,777.77 C
Should you have any queries, please do not hesitate to contact our Call Centre at 1800 226 6121 (Within Singapore) / +65 6226 6121 (Outside Singapore).
Audit Confirmation Team Cash Management Operations Centre
This is a computer generated information and no further signature is required.
IMPORTANT
The information provided h

Processing PDFs: 100%|█████████████████████████████████████████████████████████████████| 8/8 [05:39<00:00, 42.45s/file]

Raw Content:  UOB
ERNST & YOUNG LEVEL 18 NORTH TOWER ONE RAFFLES QUAY SINGAPORE 048583
Private And Confidential
United Overseas Bank Limited Cash Management Operations Centre Audit Confirmation Robinson Road PO Box 628 Singapore 901228 www.jobgroup.com
Your Ref.
Our Ref.
Our Reply Date
ABC 123 456
03-01-2024
Dear Sir,
Confirmation of Balances
EFG COMPANY
We are pleased to provide the following information as at 31-12-2023
Current Account
222-000-222
: 99,999.00 C
Should you have any queries, please do not hesitate to contact our Call Centre at 1800 226 6121 (Within Singapore) / +65 6226 6121 (Outside Singapore).
Audit Confirmation Team Cash Management Operations Centre
This is a computer generated information and no further signature is required.
IMPORTANT
The information provided herein is given on the basis that it is to be used for purposes of confirmation of our customer's records only and not as an original of conclusive source of information. While we have taken every care to pro